# Combinatorial Species and Mathematical Biology

## Trees

Recall the basic species.

* $1$ is the empty set species

* $X$ is the singleton species

* $E$ is the set species

### Rooted trees

Let's try to define the species $\mathcal A$ of rooted trees on $n$ nodes. Each tree consists of a single **root**, and a **set** of root subtrees. In the "functional equation" language of species this becomes

$$
\mathcal A = X \cdot E(\mathcal A).
$$


In [4]:
from sage.combinat.species.library import *

One = EmptySetSpecies()
X = SingletonSpecies()
E = SetSpecies()

A = CombinatorialSpecies(min=1)
A.define(X * E(A))

This allows us to perform labeled and unlabeled enumeration.

In [5]:
As = A.generating_series()

print('generating series:')
print(As)

Ac = [factorial(i)*As.coefficient(i) for i in range(1,11)]

print('counts of labeled structures:')
print(Ac)

Ais = A.isotype_generating_series()

print('isotype generating series:')
print(Ais)

Au = Ais[:11]

print('counts of unlabeled structures:')
print(Au)

generating series:
z + z^2 + 3/2*z^3 + 8/3*z^4 + 125/24*z^5 + 54/5*z^6 + 16807/720*z^7 + O(z^8)
counts of labeled structures:
[1, 2, 9, 64, 625, 7776, 117649, 2097152, 43046721, 1000000000]
isotype generating series:
z + z^2 + 2*z^3 + 4*z^4 + 9*z^5 + 20*z^6 + 48*z^7 + O(z^8)
counts of unlabeled structures:
[1, 1, 2, 4, 9, 20, 48, 115, 286, 719]


In [6]:
oeis(Ac)[0]

A000169: Number of labeled rooted trees with n nodes: n^(n-1).

In [7]:
oeis(Au)[0]

A000081: Number of unlabeled rooted trees with n nodes (or connected functions with a fixed point).

Searching OEIS gives us the right sequence IDs as the first option. In particular, the number of rooted labeled trees is $n^{n-1}$.

### Binary rooted trees

Now let's try planar binary trees $\mathcal B$. There is one tree on the empty set of labels, and otherwise a tree is a **root** node, together with a **left subtree** and the **right subtree**

In [8]:
B = CombinatorialSpecies()
B.define(One + X * B * B)

In [6]:
B.isotype_generating_series()

1 + z + 2*z^2 + 5*z^3 + 14*z^4 + 42*z^5 + 132*z^6 + O(z^7)

In [7]:
oeis([1,1,2,5,14,42])

0: A000108: Catalan numbers: C(n) = binomial(2n,n)/(n+1) = (2n)!/(n!(n+1)!).
1: A120588: G.f. is 1 + x*c(x), where c(x) is the g.f. of the Catalan numbers (A000108).
2: A080937: Number of Catalan paths (nonnegative, starting and ending at 0, step +/-1) of 2*n steps with all values <= 5.

## Phylogenetic trees

But what about phylogenetic trees? The leaves are labeled but the internal nodes aren't.

We can solve this:

$$ \mathcal T = X + E_2(\mathcal T),$$

where $E_2$ is a species of pairs. Intuitively, a tree here is either a leaf or an unlabeled root together with a pair of subtrees.

In [20]:
E = SetSpecies()
E2 = E.restricted(min=2, max=3)

T = CombinatorialSpecies(min=1)
T.define(X + E2(T))

In [21]:
Tis = T.isotype_generating_series()

print('isotype generating series:')
print(Tis)

Tu = Tis[:17]

print('counts of unlabeled structures:')
print(Tu)

isotype generating series:
z + z^2 + z^3 + 2*z^4 + 3*z^5 + 6*z^6 + 11*z^7 + O(z^8)
counts of unlabeled structures:
[1, 1, 1, 2, 3, 6, 11, 23, 46, 98, 207, 451, 983, 2179, 4850, 10905]


In [22]:
oeis(Tu)

0: A001190: Wedderburn-Etherington numbers: unlabeled binary rooted trees (every node has outdegree 0 or 2) with n endpoints (and 2n-1 nodes in all).

By the way, the cycle index series is implicitly used in this calculation:

In [23]:
T.cycle_index_series()

p[1] + (1/2*p[1,1]+1/2*p[2]) + (1/2*p[1,1,1]+1/2*p[2,1]) + (5/8*p[1,1,1,1]+3/4*p[2,1,1]+3/8*p[2,2]+1/4*p[4]) + (7/8*p[1,1,1,1,1]+5/4*p[2,1,1,1]+5/8*p[2,2,1]+1/4*p[4,1]) + (21/16*p[1,1,1,1,1,1]+35/16*p[2,1,1,1,1]+21/16*p[2,2,1,1]+7/16*p[2,2,2]+3/8*p[4,1,1]+3/8*p[4,2]) + (33/16*p[1,1,1,1,1,1,1]+63/16*p[2,1,1,1,1,1]+45/16*p[2,2,1,1,1]+15/16*p[2,2,2,1]+5/8*p[4,1,1,1]+5/8*p[4,2,1]) + O^8

Any trees can be defined like this. For example, here is the multifurcating tree species:

In [42]:
E2p = E.restricted(min=2)

Tmult = CombinatorialSpecies(min=1)
Tmult.define(X + E2p(Tmult))

In [25]:
# lookup in OEIS 
oeis(Tmult.isotype_generating_series()[:10])

0: A000669: Number of series-reduced planted trees with n leaves. Also the number of essentially series series-parallel networks with n edges; also the number of essentially parallel series-parallel networks with n edges.
1: A292215: Number of (unlabeled) rooted trees with n leaf nodes and without unary nodes or outdegrees larger than nine.
2: A292216: Number of (unlabeled) rooted trees with n leaf nodes and without unary nodes or outdegrees larger than ten.

## Tanglegrams

The 

In [35]:
def cartprod_isotypes(F, G, n=17):
    # extract cycle index series Z_F, G_F
    ZF = F.cycle_index_series()
    ZG = G.cycle_index_series()
    
    result = []
    
    # for terms of each degree,
    # extract the coefficients r_lambda and multiply them together
    for deg in range(1,n):
        
        val = 0
        
        # each partition corresponds to a monomial in the cycle index series
        for p_lambda in Partitions(deg).list():
            z_lambda = Partition(p_lambda).aut()
            
            r_lambda_F = ZF.coefficient(deg).coefficient(p_lambda) * z_lambda
            r_lambda_G = ZG.coefficient(deg).coefficient(p_lambda) * z_lambda
            
            val += (r_lambda_F * r_lambda_G) / z_lambda

        result.append(val)
        
    return(result)

In [36]:
cartprod_isotypes(T,T)

[1,
 1,
 2,
 13,
 114,
 1509,
 25595,
 535753,
 13305590,
 382728552,
 12515198465,
 458621603279,
 18619063906689,
 829607273337513,
 40253392454978755,
 2112878091130119496]

In [37]:
oeis(cartprod_isotypes(T,T))

0: A258620: Number of tanglegrams of size n.

In [38]:
cartprod_isotypes(Tmult,Tmult)

[1,
 1,
 5,
 51,
 757,
 16416,
 461231,
 16021550,
 662197510,
 31749450007,
 1732478051823,
 106025572201434,
 7192665669790893,
 535756912504764218,
 43471544417828923777,
 3816784803681841133512]

In [39]:
oeis(cartprod_isotypes(Tmult,Tmult))

0: A339234: Number of series-reduced tanglegrams with n unlabeled leaves.

## Bonus: a different problem



In [43]:
E1p = E.restricted(min=1)

sp = CombinatorialSpecies()
sp.define(E + E*E1p(E2p))

In [47]:
fi = sp.isotype_generating_series()
fi

1 + z + 2*z^2 + 3*z^3 + 5*z^4 + 7*z^5 + 11*z^6 + O(z^7)

In [48]:
oeis(fi[:10])

0: A000041: a(n) is the number of partitions of n (the partition numbers).
1: A332280: Number of integer partitions of n with unimodal run-lengths.
2: A026815: Number of partitions of n in which the greatest part is 9.

In [49]:
f = sp.generating_series()
f

1 + z + z^2 + 5/6*z^3 + 5/8*z^4 + 13/30*z^5 + 203/720*z^6 + O(z^7)

In [53]:
fcoef = [x*factorial(i+1) for i, x in enumerate(sp.generating_series()[1:16])]
fcoef

[1,
 2,
 5,
 15,
 52,
 203,
 877,
 4140,
 21147,
 115975,
 678570,
 4213597,
 27644437,
 190899322,
 1382958545]

In [54]:
oeis(fcoef)

0: A000110: Bell or exponential numbers: number of ways to partition a set of n labeled elements.
1: A203646: Number of arrays of n 0..15 integers with new values introduced in order 0..15 but otherwise unconstrained.
2: A203645: Number of arrays of n 0..14 integers with new values introduced in order 0..14 but otherwise unconstrained.